# **Proyecto Final**


## **Creamos la sesion de spark**


___________

In [ ]:
# 📌 1️⃣ Eliminar cualquier instalación previa
!rm -rf /content/spark-*
!apt-get update > /dev/null
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# 📌 2️⃣ Descargar Spark 3.5.0 desde Apache (con verificación de descarga)
!curl -o /content/spark-3.5.0-bin-hadoop3.tgz https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

# 📌 3️⃣ Verificar si el archivo se descargó correctamente
!ls -lh /content/spark-3.5.0-bin-hadoop3.tgz

# 📌 4️⃣ Extraer el archivo si es válido
!tar xf /content/spark-3.5.0-bin-hadoop3.tgz -C /content/

# 📌 5️⃣ Configurar variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["SPARK_HOME"], "bin")
os.environ["PYTHONPATH"] = os.path.join(os.environ["SPARK_HOME"], "python") + os.pathsep + os.path.join(os.environ["SPARK_HOME"], "python/lib/py4j-0.10.9.7-src.zip")

# 📌 6️⃣ Instalar PySpark
!pip install -q pyspark

# 📌 7️⃣ Iniciar la sesión de Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MiSparkApp") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print("✅ Spark inicializado correctamente:", spark.version)


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  381M  100  381M    0     0   223k      0  0:29:09  0:29:09 --:--:--  192k
-rw-r--r-- 1 root root 382M Mar  4 10:01 /content/spark-3.5.0-bin-hadoop3.tgz
✅ Spark inicializado correctamente: 3.5.0


In [ ]:
# importamos funciones para trabajo con dataframes
from pyspark.sql import functions as F


____________________
_______________

## **Cargamos los archivos limpios de la fase anterior**

______________

In [ ]:
df_empresas= spark.read.parquet('/content/Empresas_limpio.parquet', inferSchema=True)
df_empresas.show(5)

+----------+---------+---------------+-------------+----------------------+----------+-------+
|empresa_id|   nombre|consumo_energia|emisiones_co2|certificacion_iso14001|    sector|   pais|
+----------+---------+---------------+-------------+----------------------+----------+-------+
| EMP000000|Empresa_0|        44404.7|       3217.3|                     0|   Energia| Canada|
| EMP000001|Empresa_1|       25576.11|      16135.7|                     1|  Finanzas|  China|
| EMP000002|Empresa_2|       29305.07|      2224.93|                     0|     Salud|  Japon|
| EMP000003|Empresa_3|       44600.25|      3764.32|                     1|Transporte|  China|
| EMP000004|Empresa_4|       46039.63|     11840.17|                     1|Tecnologia|Francia|
+----------+---------+---------------+-------------+----------------------+----------+-------+
only showing top 5 rows



In [ ]:
df_proyectos= spark.read.parquet('/content/proyectos_limpio.parquet', inferSchema=True)
df_proyectos.show(5)

+-----------+----------+--------------------+-------------------+--------------+---------------+------------+
|proyecto_id|empresa_id|capacidad_generacion|reduccion_emisiones|costo_proyecto|estado_proyecto|tipo_energia|
+-----------+----------+--------------------+-------------------+--------------+---------------+------------+
| PROJ000000| EMP016520|             9131.06|            2270.88|         26.03|         Activo|       Solar|
| PROJ000001| EMP012923|             2164.32|             4886.1|        364.36|     Finalizado|       Solar|
| PROJ000002| EMP023136|             2374.24|              446.3|        190.17|  En desarrollo|     Biomasa|
| PROJ000003| EMP004436|             4065.11|            4465.83|         126.6|     Finalizado|  Geotermica|
| PROJ000004| EMP021681|             4581.51|            3280.46|        227.13|         Activo|      Eolica|
+-----------+----------+--------------------+-------------------+--------------+---------------+------------+
only showi

In [ ]:
df_regulaciones= spark.read.parquet('/content/Regulaciones_limpio.parquet', inferSchema=True)
df_regulaciones.show(5)

+-------------+----------------+--------------------+----------------+--------+
|regulacion_id|limite_emisiones|subsidios_renovables|impuesto_carbono|    pais|
+-------------+----------------+--------------------+----------------+--------+
|    REG000000|         3012.63|                   1|           19.54|   India|
|    REG000001|         7325.54|                   1|           42.41|    EEUU|
|    REG000002|         9060.84|                   1|            6.15|Alemania|
|    REG000003|         7075.95|                   1|           44.65|  Espana|
|    REG000004|         1997.42|                   0|            8.75|Alemania|
+-------------+----------------+--------------------+----------------+--------+
only showing top 5 rows



______________
______________

## **Tabla Eficiencia Energetica Empresas**

_________

In [ ]:
# calculamos la primera columna que ira en la tabla de salida.
df_consumo_energia= df_empresas.groupBy('empresa_id').agg(F.sum('consumo_energia').alias('consumo_energia_total'))
df_consumo_energia.sort(F.col('empresa_id')).show(5)
df_consumo_energia.groupBy().count().show()

+----------+---------------------+
|empresa_id|consumo_energia_total|
+----------+---------------------+
| EMP000000|              44404.7|
| EMP000001|             25576.11|
| EMP000002|             29305.07|
| EMP000003|             44600.25|
| EMP000004|             46039.63|
+----------+---------------------+
only showing top 5 rows

+-----+
|count|
+-----+
|24627|
+-----+



In [ ]:
df_empresas.groupBy('nombre').count().filter(F.col('count')>1).show(5)

+------+-----+
|nombre|count|
+------+-----+
+------+-----+



#### Observaciones:
- Agrupamos por empresa id y vemos que tenemos el total de los registros del data frame inicial, lo que quiere decir que el consumo total ya viene dado por el dataframe inicial de las empresas, sin necesidad de la agrupación, ya que no hay ID repetidos por los cuales agrupar.

- Verifico si agrupando por nombre hay algun nombre de empresa que se repite, ya que podría tener un mismo nombre asignado a dos ID y generaria inconsistencias, en dado caso agruparia por empresa id y nombre para tratarlos como registros distintos. Sin embargo no se tienen empresas repetidas. Puedo trabajar el consumo total como el que viene dado para cada empresa en el archivo original.

In [ ]:
# Pasamos a calcular el segundo campo de la tabla salida, Energia renovable que se refiere a la capacidad generadora por empresa
df_energia_renovable=  df_proyectos.groupBy('empresa_id').agg(F.round(F.sum('capacidad_generacion'),2).alias('energia_renovable'))
df_energia_renovable.sort(F.col('empresa_id')).show(5)
df_energia_renovable.groupBy().count().show()

+----------+-----------------+
|empresa_id|energia_renovable|
+----------+-----------------+
| EMP000000|         11112.99|
| EMP000001|          7295.54|
| EMP000004|           948.21|
| EMP000005|          9439.39|
| EMP000007|         13240.99|
+----------+-----------------+
only showing top 5 rows

+-----+
|count|
+-----+
|16033|
+-----+



In [ ]:
df_proyectos.filter(F.col('empresa_id')=='EMP000000').show()

+-----------+----------+--------------------+-------------------+--------------+---------------+--------------+
|proyecto_id|empresa_id|capacidad_generacion|reduccion_emisiones|costo_proyecto|estado_proyecto|  tipo_energia|
+-----------+----------+--------------------+-------------------+--------------+---------------+--------------+
| PROJ002488| EMP000000|             9150.33|             224.96|        301.21|         Activo|Hidroelectrica|
| PROJ020806| EMP000000|             1962.66|             606.28|        375.11|  En desarrollo|        Eolica|
+-----------+----------+--------------------+-------------------+--------------+---------------+--------------+



#### Observaciones:

- En este caso agrupamos por empresa ID y sumamos la capacidad de generación de cada una de ellas en sus distintos proyectos. Esto seria la energia renovable que generan.

- Observamos que la agrupación en este caso si tiene sentido, ya que una empresa puede tener multiples proyectos, y debido a que me interesa el total de la energia renovable que generan, la obtengo con el groupby. La empresa con ID EMP000000, ejemplifica el objetivo de esta agrupación. Tambien podemos ver que originalmente teniamos 25808 proyectos y cuando agrupamos por empresa se tienen 16033 empresas distintas cada una con x cantidad de proyectos que sumaran en total los 25808 mencionados.

In [ ]:
# Para poder calcular el resto de campos referentes a la tabla 1, debemos cruzar la información obtenida en los dos pasos anteriores.

df_consumo_X_energia_renovable= df_empresas.join(df_energia_renovable, on='empresa_id', how='inner').select('empresa_id','nombre','consumo_energia','energia_renovable')
df_consumo_X_energia_renovable= df_consumo_X_energia_renovable.withColumnRenamed('consumo_energia', 'consumo_energia_total')
df_consumo_X_energia_renovable.sort('empresa_id').show(5)
df_consumo_X_energia_renovable.groupBy().count().show()


+----------+---------+---------------------+-----------------+
|empresa_id|   nombre|consumo_energia_total|energia_renovable|
+----------+---------+---------------------+-----------------+
| EMP000000|Empresa_0|              44404.7|         11112.99|
| EMP000001|Empresa_1|             25576.11|          7295.54|
| EMP000004|Empresa_4|             46039.63|           948.21|
| EMP000005|Empresa_5|             41988.85|          9439.39|
| EMP000007|Empresa_7|             25219.96|         13240.99|
+----------+---------+---------------------+-----------------+
only showing top 5 rows

+-----+
|count|
+-----+
|16033|
+-----+



#### Observaciones:

- Como se habia comentado antes, el consumo de energia total ya viene dado en el archivo inicial de empresas, por lo tanto cruzamos este dataframe con el agrupado en el paso anterior que incluye la energia renovable generada por empresa. El cruce es de tipo inner para quedarnos con la intersección, en este caso se conservan la misma cantidad de registros que se tiene en el dataframe de energias renovables. Es decir se quedan por fuera del cruce unicamente las empresas que no tienen proyectos renovables o que tienen proyectos y no aparecen en el dataframe de empresas.

- La salida nos muestra que tenemos en el cruce inner el mismo total de registros de la agrupacion de energia renovables por empresa, es decir que las unicas empresas que quedaron fuera del cruce son las que hacen parte del archivo de empresas y aparecen sin proyectos renovables, ya que al tener un cruce inner que me esta regresando el total de registros del Dataframe de energias renovables y no tener duplicados en ninguno de los dos Dataframes, se puede afirmar que contiene todos los registros de dicho dataframe de energia renovable.

- Las empresas que no aparecen en el archivo de proyectos, no tendria sentido incluirlas en el dataset de salida, ya que todos los campos van relacionados con la energia renovable por lo que tendrian valores nulos en todas las columnas. Sin embargo es información valiosa saber cuales son estas empresas que no aparecen con proyectos renovables, por lo cual al exportar el dataset de salida, se puede realizar un cruce leftanti del dataframe de empresas con el datafrane que tendremos como salida, para exportar este archivo aparte y brindar tanto la información requerida por el proceso, como el complemento con la información de esas empresas al parecer sin proyectos.

In [ ]:
# Pasamos a calcular ahora si el tercer campo, que seria la tasa porcentual entre los dos campos calculados anteriormente.
df_porcentaje_renovable= df_consumo_X_energia_renovable.withColumn('porcentaje_renovable', F.round((F.col('energia_renovable')/F.col('consumo_energia_total'))*100,2))
df_porcentaje_renovable.sort('empresa_id').show(5)

+----------+---------+---------------------+-----------------+--------------------+
|empresa_id|   nombre|consumo_energia_total|energia_renovable|porcentaje_renovable|
+----------+---------+---------------------+-----------------+--------------------+
| EMP000000|Empresa_0|              44404.7|         11112.99|               25.03|
| EMP000001|Empresa_1|             25576.11|          7295.54|               28.52|
| EMP000004|Empresa_4|             46039.63|           948.21|                2.06|
| EMP000005|Empresa_5|             41988.85|          9439.39|               22.48|
| EMP000007|Empresa_7|             25219.96|         13240.99|                52.5|
+----------+---------+---------------------+-----------------+--------------------+
only showing top 5 rows



In [ ]:
df_porcentaje_renovable.select(
    F.col("empresa_id"),
    F.col("nombre"),
    F.col("consumo_energia_total"),
    F.col("energia_renovable"),
    F.format_string("%.2f%%", F.col("porcentaje_renovable")).alias("porcentaje_renovable")
).sort('empresa_id').show(5)

+----------+---------+---------------------+-----------------+--------------------+
|empresa_id|   nombre|consumo_energia_total|energia_renovable|porcentaje_renovable|
+----------+---------+---------------------+-----------------+--------------------+
| EMP000000|Empresa_0|              44404.7|         11112.99|              25.03%|
| EMP000001|Empresa_1|             25576.11|          7295.54|              28.52%|
| EMP000004|Empresa_4|             46039.63|           948.21|               2.06%|
| EMP000005|Empresa_5|             41988.85|          9439.39|              22.48%|
| EMP000007|Empresa_7|             25219.96|         13240.99|              52.50%|
+----------+---------+---------------------+-----------------+--------------------+
only showing top 5 rows



#### Obvservaciones:

- El campo de porcentaje renovable se calcula como la tasa entre al energia renivable generada por una empresa sobre su consumo de energia total. Multiplicamos esto por 100, y lo redondeamos a dos decimales para tener el formato porcentual, por ejemplo 100.00%, en caso de que una empresa consuma y genere exactamente lo mismo.

- El valor se guarda como double con dos decimales en cada registro, sin embargo para facilitar la visualización del calculo, mostramos en pantalla con un formato que incluye el porcentaje, el calculo realizado. No se guarda el valor con el signo %, ya que para hacer esto tendriamos que convertir los registros a tipo String, por lo cual unicamente los visualizamos una vez.

In [ ]:
# Procedemos a calular el 4to campo, que seria el indice entre energia renovable y emisiones de carbono.
df_emisiones_co2= df_empresas.select('empresa_id','emisiones_co2')
df_eficiencia_energetica= df_porcentaje_renovable.join(df_emisiones_co2, on='empresa_id', how='left')

df_eficiencia_energetica=  df_eficiencia_energetica.withColumn('eficiencia_energetica', F.round(F.col('energia_renovable')/F.col('emisiones_co2'),2))
df_eficiencia_energetica= df_eficiencia_energetica.select(
    'empresa_id','nombre','consumo_energia_total','energia_renovable','porcentaje_renovable','eficiencia_energetica')

df_eficiencia_energetica.sort('empresa_id').show(5)


+----------+---------+---------------------+-----------------+--------------------+---------------------+
|empresa_id|   nombre|consumo_energia_total|energia_renovable|porcentaje_renovable|eficiencia_energetica|
+----------+---------+---------------------+-----------------+--------------------+---------------------+
| EMP000000|Empresa_0|              44404.7|         11112.99|               25.03|                 3.45|
| EMP000001|Empresa_1|             25576.11|          7295.54|               28.52|                 0.45|
| EMP000004|Empresa_4|             46039.63|           948.21|                2.06|                 0.08|
| EMP000005|Empresa_5|             41988.85|          9439.39|               22.48|                 1.67|
| EMP000007|Empresa_7|             25219.96|         13240.99|                52.5|                  1.4|
+----------+---------+---------------------+-----------------+--------------------+---------------------+
only showing top 5 rows



#### Observaciones:

- Necesitamos calcular el indice entre energia renovable que tenemos en el data frame de porcentaje renovable, y emisiones co2 que esta en el original de empresas. Como sabemos que ya los valores estan totalizados en ese dataframe, unicamente sacamos las columnas empresa id y emisiones co2 para poder cruzar y traer la varibale emisiones al dataframe que hemos ido construyendo.

- Realizamos un cruce left entre el dataframe que tenemos hasta el calculo de la ultima varibale porcentaje renivable y el dataframe contruido para tener las emisiones de co2, podria ser inner y daria igual ya que como vimos anteriormente las unicas empresas que queda por fuera son las del lado del archivo original de empresas que no tienen proyectos renovables.

- Una vez realizado el cruce, dividimos los campos de interes, en este caso es un indice asi que no multiplicamos por 100, sino que dejamos el valor numerico tal cual.

- Por ultimo sobre el mismo dataframe que trabjamos el cruce y calculamos la nueva columna, seleccionamos todas menos la de emsiones co2, ya que esta no va en la tabla de salida. Teniendo esto, ya podemos cosntruir sobre este Dataframe, la ultima columna de nuestra tabla salida en el siguiente paso.

In [ ]:
# Procedemos a añadir nuestra ultima columna de la primera tabla.
df_empresa_sostenible= df_eficiencia_energetica.withColumn('empresa_sostenible', F.when((F.col('porcentaje_renovable')>50) & (F.col('eficiencia_energetica')>0.5), 'Si').otherwise('No'))
df_empresa_sostenible.sort('empresa_id').show(5)

+----------+---------+---------------------+-----------------+--------------------+---------------------+------------------+
|empresa_id|   nombre|consumo_energia_total|energia_renovable|porcentaje_renovable|eficiencia_energetica|empresa_sostenible|
+----------+---------+---------------------+-----------------+--------------------+---------------------+------------------+
| EMP000000|Empresa_0|              44404.7|         11112.99|               25.03|                 3.45|                No|
| EMP000001|Empresa_1|             25576.11|          7295.54|               28.52|                 0.45|                No|
| EMP000004|Empresa_4|             46039.63|           948.21|                2.06|                 0.08|                No|
| EMP000005|Empresa_5|             41988.85|          9439.39|               22.48|                 1.67|                No|
| EMP000007|Empresa_7|             25219.96|         13240.99|                52.5|                  1.4|                Si|


#### Observaciones:

- Teniendo los campos de porcentaje renovable y eficiencia energetica, unicamente se añade una columna en la que tendremos que una empresa es sostenible o no, dependiendo si cumple las condiciones dadas, que serian tener un porcentaje renivable mayor al 50% y un indice en la eficiencia energetica mayor a 0.5

In [ ]:
df_empresas_sin_proyectos_sostenibles= df_empresas.join(df_empresa_sostenible, on='empresa_id', how='leftanti')
df_empresas_sin_proyectos_sostenibles.sort('empresa_id').show(5)
df_empresas_sin_proyectos_sostenibles.groupBy().count().show()

+----------+----------+---------------+-------------+----------------------+-----------+------+
|empresa_id|    nombre|consumo_energia|emisiones_co2|certificacion_iso14001|     sector|  pais|
+----------+----------+---------------+-------------+----------------------+-----------+------+
| EMP000002| Empresa_2|       29305.07|      2224.93|                     0|      Salud| Japon|
| EMP000003| Empresa_3|       44600.25|      3764.32|                     1| Transporte| China|
| EMP000006| Empresa_6|        2793.22|      3871.34|                     0|    Energia| China|
| EMP000009| Empresa_9|        6868.81|      13398.5|                     1|    Energia| China|
| EMP000012|Empresa_12|       24567.02|      1807.39|                     0|Manufactura|Canada|
+----------+----------+---------------+-------------+----------------------+-----------+------+
only showing top 5 rows

+-----+
|count|
+-----+
| 8594|
+-----+



In [ ]:
df_eficiencia_energetica_empresas= df_empresa_sostenible

df_eficiencia_energetica_empresas.write.csv('/content/Eficiencia_Energetica_Empresas.csv', header=True, mode='overwrite')

df_empresas_sin_proyectos_sostenibles.write.csv('/content/Empresas_sin_proyectos_sostenibles.csv', header=True, mode='overwrite')

#### Observaciones:

- Teniendo ya la tabla de salida depurada, almaceno el ultimo Data frame exactamente igual en otro Dataframe llamado como la tabla de salida para distinguirlo como la salida de mi proceso y lo exporto.

- Como se menciono antes, tambien exportamos las empresas que no tienen proyectos sostenibles aparte. Ya que al no tener energia renovable registrada, tendriamos valores nulos en 4 de los 5 campos de la tabla salida, sin embargo puede ser información valiosa distinguir cuales son estas empresas que no registran proyectos. Comprobamos que tenemos 8594 empresas, que sumadas a las 16033 que si tienen, nos da el total de 24627 empresas del archivo inicial, por lo cual se ha cruzado bien.

_____
______